In [ ]:
import os

In [ ]:
def get_info_by_folder(folder_name):
    info = {
        "function": ("函式封裝練習", "參數傳遞、Return 回傳值應用"),
        "loop": ("迴圈演算法練習", "for/while 迭代、控制流程"),
        "if_else": ("邏輯判斷練習", "條件分歧、布林邏輯運算"),
        "list": ("資料結構與排序", "串列操作、演算法效率優化"),
        "variable_constant": ("基礎語法練習", "變數賦值、基本數學運算"),
        "utils": ("工具程式", "專案輔助腳本與自動化工具")  # 新增這一行
    }
    return info.get(folder_name, ("基礎 Python 練習", "語法熟悉與邏輯實作"))

In [ ]:
def update_files():
    # 這裡加入 utils 資料夾
    target_folders = ["function", "loop", "if_else", "list", "variable_constant", "utils"]
    
    for folder in target_folders:
        if not os.path.exists(folder):
            continue
            
        for file in os.listdir(folder):
            # 處理 .py 且不處理腳本自己 (避免自己幫自己無限加註解)
            if file.endswith(".py") and file != "add_docstrings.py":
                file_path = os.path.join(folder, file)
                
                feature, point = get_info_by_folder(folder)
                docstring = f'"""\n檔名：{file}\n功能：{feature}\n學習重點：{point}\n"""\n'
                
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                
                if not content.startswith('"""'):
                    with open(file_path, "w", encoding="utf-8") as f:
                        f.write(docstring + content)
                    print(f"✅ 已完成標註: [{folder}] {file}")

In [ ]:
if __name__ == "__main__":
    update_files()